# Generating C++ Kokkos Chemistry Solvers with JAFF

This notebook demonstrates how to use JAFF to generate C++ code that uses the Kokkos-kernels BDF solver for solving chemical reaction networks. We'll use the `react_COthin` network as an example, which contains 37 species.

## Overview

The Kokkos output plugin generates:
- **chemistry_ode.hpp**: Chemistry ODE system class with evaluate_function and evaluate_jacobian methods
- **chemistry_ode.cpp**: Main executable that sets up and runs the BDF solver
- **CMakeLists.txt**: Build configuration for Kokkos and Kokkos-kernels dependencies

The generated code is compatible with the Kokkos-kernels BDF solver for stiff ODEs.

## Step 1: Load the Chemical Network

First, let's load the react_COthin network, which is a carbon-oxygen chemistry network in KROME format.

In [1]:
from jaff.network import Network
from jaff.builder import Builder
import os

# Load the test2 network
network = Network("../networks/react_COthin")

print(f"Network loaded: {network.label}")
print(f"Number of species: {network.get_number_of_species()}")
print(f"Number of reactions: {len(network.reactions)}")

Welcome to JAFF: Just Another Fancy Format!
Loading network from ../networks/react_COthin
Network label = react_COthin


  3%|██▋                                                                                    | 11/350 [00:00<00:03, 95.79it/s]

KROME variable detected: @var:Hnuclei = get_Hnuclei(n(:))
KROME variable detected: @var: Te = Tgas*8.617343d-5
KROME variable detected: @var: invT = 1d0/Tgas
KROME variable detected: @var: lnTe = log(Te)
KROME variable detected: @var: T = Tgas
KROME format detected: @format:idx,R,R,R,P,P,P,P,Tmin,Tmax,rate
exp(-2.03914985e-6*lnte**8 + 0.000111954395*lnte**7 - 0.00263197617*lnte**6 + 0.0348255977*lnte**5 - 0.2877056*lnte**4 + 1.56315498*lnte**3 - 5.73932875*lnte**2 + 13.536556*lnte - 32.71396786)
3.92e-13*invte**0.6353
exp(-3.071135243196595e-9*lnte**9 - 1.856767039775261e-8*lnte**8 + 5.755614137575758e-7*lnte**7 + 4.989108920299513e-6*lnte**6 - 1.421502914054107e-5*lnte**5 - 0.0003212605213188796*lnte**4 - 0.002380861877349834*lnte**3 - 0.02026044731984691*lnte**2 - 0.7241125657826851*lnte - 28.61303380689232)
exp(-3.64916141e-6*lnte**8 + 0.000206723616*lnte**7 - 0.0050090561*lnte**6 + 0.0679539123*lnte**5 - 0.56851189*lnte**4 + 3.05803875*lnte**3 - 10.7532302*lnte**2 + 23.91596563*lnt

 11%|█████████▌                                                                            | 39/350 [00:00<00:03, 102.53it/s]

7.20000000000000e-15
KROME format detected: @format:idx,R,R,R,P,P,P,Tmin,Tmax,rate
3.7e-14*exp(35.0*invt)
3.0e-11*sqrt(t32)*exp(-52000.0*invt)
3.36e-10/(sqrtgas*((tgas*1.0e-6)**0.7 + 1.0)*((tgas*0.001)**0.2))
6.77e-15*te**0.8779
1.43000000000000e-9
exp(2.238306228891639e-7*lnte**7 - 8.36671960467864e-6*lnte**6 + 0.0001073294010367247*lnte**5 - 0.0003105115447124016*lnte**4 - 0.004555120027032095*lnte**3 + 0.03599837721023835*lnte**2 + 0.2289800603272916*lnte - 20.06913897587003)
1.85e-23*tgas**1.8
5.81e-16/(tgas/56200.0)**(0.6657*log10(tgas/56200.0))
6.00000000000000e-10
exp(-9.36345888928611e-6*lnte**8 + 0.0004138398421504563*lnte**7 - 0.007879026154483455*lnte**6 + 0.0841077503763412*lnte**5 - 0.5416182856220388*lnte**4 + 2.045587822403071*lnte**3 - 3.898003964650152*lnte**2 + 3.400824447095291*lnte - 24.24914687731536)
5.6e-11*tgas**0.5*exp(-102124.0*invt)
1.0670825e-10*te**2.012*exp(-4.463*invte)/((0.2472*te + 1.0)**3.512)
exp(-2.631285809207e-6*lnte**8 + 0.0001068275202678*lnte**7

...

2.28000000000000e-8
9.15e-10*(2.62185*(300.0*invt)**0.5 + 0.62)
1.01000000000000e-9
6.40000000000000e-10
2.55000000000000e-8
1.73e-9*(2.578947*(300.0*invt)**0.5 + 0.62)
1.80000000000000e-9
5.03000000000000e-9
3.4093e-10*(2.578947*(300.0*invt)**0.5 + 0.62)
2.40000000000000e-10
1.00000000000000e-11
3.42000000000000e-10
4.53000000000000e-10
9.10000000000000e-10
5.20000000000000e-11
5.20000000000000e-11
2.47000000000000e-9
1.88055e-9*(1.79558*invt + 0.02427*(300.0*invt)**0.5 + 1.0)
1.45000000000000e-10
1.08256e-10*(1.79558*invt + 0.02427*(300.0*invt)**0.5 + 1.0)
1.00000000000000e-9
1.23000000000000e-8
8.34e-10*(834.16588*invt + 0.5232*(300.0*invt)**0.5 + 1.0)
3.30000000000000e-8
3.54e-9*(1.587411*(300.0*invt)**0.5 + 0.62)
1.765e-9*(0.672147*(300.0*invt)**0.5 + 0.62)
1.765e-9*(56.66255*invt + 0.136347*(300.0*invt)**0.5 + 1.0)
1.765e-9*(0.672147*(300.0*invt)**0.5 + 0.62)
1.765e-9*(56.66255*invt + 0.136347*(300.0*invt)**0.5 + 1.0)
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
9.

 80%|████████████████████████████████████████████████████████████████████▏                | 281/350 [00:01<00:00, 424.43it/s]

7.68e-8/t32**0.6
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
4.03e-7/t32**0.6
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
7.75e-8/t32**0.5
1.95e-7/t32**0.5
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
2.0e-7/t32**0.5
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
6.3e-9/t32**0.48
3.9e-8/t32**0.5
8.6e-8/t32**0.5
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
3.05e-7/t32**0.5
2.58e-7/t32**0.5
5.6e-9/t32**0.5
KROME format detected: @format:idx,R,R,P,P,Tmin,Tmax,rate
1.08e-7/t32**0.5
6.02e-7/t32**0.5
1.95e-7/t32**0.7
2.75e-7/t32**0.55
2.76e-7/t32**0.64
2.4e-8/t32**0.64
1.1e-7*invt32
1.00000000000000e-9
1.00000000000000e-10
5.00000000000000e-10
1.00000000000000e-13
5.00000000000000e-10
5.00000000000000e-10
7.00000000000000e-10
7.00000000000000e-10
5.00000000000000e-10
6.4e-10*exp(-21300.0*invt)
KROME format detected: @format:idx,R,R,P,P,P,Tmin,Tmax,rate
3.0e-11*t32**0.5*exp(-52000.0*invt)
KROME format detected: @format:idx

100%|█████████████████████████████████████████████████████████████████████████████████████| 350/350 [00:01<00:00, 276.05it/s]

6.16e-27*ntot/(tgas/300.0)**3.08
2.14e-27*ntot*exp(2114.0/tgas)/(tgas/300.0)**3.08
6.16e-27*ntot/(tgas/300.0)**3.08
2.14e-27*ntot*exp(2114.0/tgas)/(tgas/300.0)**3.08
4.33e-32*ntot/t32
2.56e-31*ntot/t32**2
9.2e-34*ntot/t32
Variables found: ['av', 'crate', 'tdust', 'tgas']
Loaded 287 reactions
Lodaded 0 photo-chemistry reactions
All done!
Network loaded: react_COthin
Number of species: 37
Number of reactions: 287


## Step 2: Examine Network Properties

Let's examine some properties of the network to understand what we're working with.

In [2]:
for i, species in enumerate(network.species):
    print(f"  {i}: {species.name} (mass: {species.mass:.2e}, charge: {species.charge})")

print(f"\nTotal species: {len(network.species)}")

  0: H (mass: 1.67e-24, charge: 0)
  1: e- (mass: 9.11e-28, charge: -1)
  2: H+ (mass: 1.67e-24, charge: 1)
  3: He (mass: 6.65e-24, charge: 0)
  4: He+ (mass: 6.65e-24, charge: 1)
  5: He++ (mass: 6.65e-24, charge: 2)
  6: H2 (mass: 3.35e-24, charge: 0)
  7: H2+ (mass: 3.35e-24, charge: 1)
  8: H- (mass: 1.67e-24, charge: -1)
  9: C+ (mass: 1.99e-23, charge: 1)
  10: C (mass: 1.99e-23, charge: 0)
  11: Si+ (mass: 4.66e-23, charge: 1)
  12: Si (mass: 4.66e-23, charge: 0)
  13: O+ (mass: 2.66e-23, charge: 1)
  14: O (mass: 2.66e-23, charge: 0)
  15: Si++ (mass: 4.66e-23, charge: 2)
  16: OH (mass: 2.82e-23, charge: 0)
  17: HOC+ (mass: 4.82e-23, charge: 1)
  18: HCO+ (mass: 4.82e-23, charge: 1)
  19: CO (mass: 4.65e-23, charge: 0)
  20: CH (mass: 2.16e-23, charge: 0)
  21: CH2 (mass: 2.33e-23, charge: 0)
  22: C2 (mass: 3.99e-23, charge: 0)
  23: HCO (mass: 4.82e-23, charge: 0)
  24: H2O (mass: 2.99e-23, charge: 0)
  25: O2 (mass: 5.31e-23, charge: 0)
  26: H3+ (mass: 5.02e-24, charge: 

In [3]:
# Show a few example reactions
for i, reaction in enumerate(network.reactions[:10]):
    reactants = " + ".join([f"{r.name}" for r in reaction.reactants])
    products = " + ".join([f"{p.name}" for p in reaction.products])
    print(f"  {i+1}: {reactants} -> {products}")
    print(f"      Rate: {reaction.rate}")
    print()

  1: H + e- -> H+ + e- + e-
      Rate: 5.90824386372651e-70*exp(-2.03914985e-6*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**8 + 0.000111954395*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**7 - 0.00263197617*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**6 + 0.0348255977*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**5 - 0.2877056*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**4 + 1.56315498*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**3 - 5.73932875*log(8.617343e-5*Max(2.73, Min(100000000.0, tgas)))**2)*Max(2.73, Min(100000000.0, tgas))**13.536556

  2: H+ + e- -> H
      Rate: 1.49810881307214e-10/Max(2.73, Min(5500.0, tgas))**0.6353

  3: H+ + e- -> H
      Rate: 3.286733702438273e-10*exp(-3.071135243196595e-9*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**9 - 1.856767039775261e-8*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**8 + 5.755614137575758e-7*log(8.617343e-5*Max(5500.0, Min(100000000.0, tgas)))**7 + 4.989108920299513e-6*log(8

## Step 3: Generate C++ Kokkos Code

Now let's use the Builder to generate C++ code using the `kokkos_ode` template.

In [4]:
# Create builder and generate Kokkos C++ code
builder = Builder(network)
builds_dir = builder.build(template="kokkos_ode")

print("\nC++ Kokkos code generated successfully!")
print(f"Generated files are located at {builds_dir}")

Building network with template: kokkos_ode
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.hpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/chemistry_ode.cpp -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.cpp
Preprocessing /Users/benwibking/amrex_codes/chemistry/jaff/src/jaff/templates/kokkos_ode/CMakeLists.txt -> /Users/benwibking/amrex_codes/chemistry/jaff/examples/CMakeLists.txt
Network built successfully using template 'kokkos_ode'.
Output files are located in: /Users/benwibking/amrex_codes/chemistry/jaff/examples

C++ Kokkos code generated successfully!
Generated files are located at /Users/benwibking/amrex_codes/chemistry/jaff/examples


## Step 4: Examine Generated Files

Let's look at the structure and content of the generated files.

In [5]:
# List generated files
generated_files = [f for f in os.listdir(builds_dir) if f.endswith(('.cpp', '.hpp', '.txt'))]

print("Generated files:")
for file in sorted(generated_files):
    filepath = os.path.join(builds_dir, file)
    size = os.path.getsize(filepath)
    print(f"  {file:20} ({size:,} bytes)")

Generated files:
  CMakeLists.txt       (2,371 bytes)
  chemistry_ode.cpp    (10,153 bytes)
  chemistry_ode.hpp    (221,597 bytes)


## Step 5: Understanding the Generated Code Structure

Let's examine key components of the generated chemistry solver.

In [6]:
# Show the ChemistryODE class definition
with open(builds_dir + "/chemistry_ode.hpp", 'r') as f:
    lines = f.readlines()

# Find the class definition
class_start = None
for i, line in enumerate(lines):
    if "struct ChemistryODE" in line:
        class_start = i
        break

if class_start:
    print("ChemistryODE class structure:")
    print("=" * 50)
    # Show class definition and methods
    brace_count = 0
    for i in range(class_start, len(lines)):
        line = lines[i].rstrip()
        print(f"{i+1:2d}: {line}")
        
        if '{' in line:
            brace_count += line.count('{')
        if '}' in line:
            brace_count -= line.count('}')
            if brace_count == 0:
                break

ChemistryODE class structure:
14: struct ChemistryODE {
15:     // Number of species in the chemical network
16:     // PREPROCESS_NUM_SPECIES
17: 
18:     static constexpr int neqs = 37;
19: 
20:     // PREPROCESS_END
21: 
22:     using state_type = std::array<integrators::Real, neqs>;
23:     using rhs_type = std::array<integrators::Real, neqs>;
24:     using jacobian_type = std::array<std::array<integrators::Real, neqs>, neqs>;
25: 
26:     // Species indices and common constants
27:     // PREPROCESS_COMMONS
28: 
29:     static constexpr int idx_h = 0;
30:     static constexpr int idx_e = 1;
31:     static constexpr int idx_hj = 2;
32:     static constexpr int idx_he = 3;
33:     static constexpr int idx_hej = 4;
34:     static constexpr int idx_hejj = 5;
35:     static constexpr int idx_h2 = 6;
36:     static constexpr int idx_h2j = 7;
37:     static constexpr int idx_hk = 8;
38:     static constexpr int idx_cj = 9;
39:     static constexpr int idx_c = 10;
40:     static constexpr

## Step 6: Building and Running Instructions

Here's how to build and run the generated C++ code:

* **Configure the build with CMake:**

In [7]:
!   cmake -S . -B build

-- Configuring done (0.6s)
-- Generating done (0.0s)
-- Build files have been written to: /Users/benwibking/amrex_codes/chemistry/jaff/examples/build


In [8]:
assert(_exit_code == 0)

* **Build with parallel compilation:**

In [9]:
!   cmake --build build -j

[1/2] Building CXX object CMakeFiles/chemistry_ode.dir/chemistry_ode.cpp.o
In file included from /Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.cpp:7:
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:1344:22: warning: unused variable 'cse834' [-Wunused-variable]
 1344 |         const double cse834 = cse154*cse52;
      |                      ^~~~~~
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:1345:22: warning: unused variable 'cse835' [-Wunused-variable]
 1345 |         const double cse835 = cse154*cse54;
      |                      ^~~~~~
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:1346:22: warning: unused variable 'cse836' [-Wunused-variable]
 1346 |         const double cse836 = cse115*nden[0]*nden[3];
      |                      ^~~~~~
/Users/benwibking/amrex_codes/chemistry/jaff/examples/chemistry_ode.hpp:1347:22: warning: unused variable 'cse837' [-Wunused-variable]
 1347 |       

In [10]:
assert(_exit_code == 0)

* **Run the chemistry solver:**

In [15]:
!   ./build/chemistry_ode --analytic-jac

Solving Chemistry ODE System (header-only integrators)
Number of species: 37
Number of reactions: 287
Time interval: [0, 3.15576e+09] seconds
Initial conditions:
  nden[0] = 1e-06
  nden[1] = 1e-10
  nden[2] = 1e-10
  nden[3] = 1e-10
  nden[4] = 1e-10
  nden[5] = 1e-10
  nden[6] = 1e-10
  nden[7] = 1e-10
  nden[8] = 1e-10
  nden[9] = 1e-10
  ... (showing first 10 species)
Jacobian: analytic

Final solution at t = 3155760000 seconds:
  nden[0] = 1.000355682e-06
  nden[1] = 4.375939391e-10
  nden[2] = 1.520578955e-10
  nden[3] = 9.999999993e-11
  nden[4] = 1.000000001e-10
  nden[5] = 1e-10
  nden[6] = 1.355716137e-10
  nden[7] = 6.093460926e-11
  nden[8] = 1.505631928e-24
  nden[9] = 1.116757205e-10
  ... (showing first 10 species)

Element/charge conservation:
  C: init=1.5000000000e-09, final=1.5000000000e-09, rel_delta=-1.7867101231e-13
  H: init=1.0030000000e-06, final=1.0030000000e-06, rel_delta=6.6018744418e-13
  He: init=3.0000000000e-10, final=3.0000000000e-10, rel_delta=4.299615

In [12]:
assert(_exit_code == 0)

* **Or, run with custom time interval (1e12 seconds):**

In [16]:
!   ./build/chemistry_ode --analytic-jac 1e15

Solving Chemistry ODE System (header-only integrators)
Number of species: 37
Number of reactions: 287
Time interval: [0, 1e+15] seconds
Initial conditions:
  nden[0] = 1e-06
  nden[1] = 1e-10
  nden[2] = 1e-10
  nden[3] = 1e-10
  nden[4] = 1e-10
  nden[5] = 1e-10
  nden[6] = 1e-10
  nden[7] = 1e-10
  nden[8] = 1e-10
  nden[9] = 1e-10
  ... (showing first 10 species)
Jacobian: analytic

Final solution at t = 1e+15 seconds:
  nden[0] = 9.957984702e-07
  nden[1] = 7.553298149e-09
  nden[2] = 7.096301507e-09
  nden[3] = 9.938716229e-11
  nden[4] = 1.006240553e-10
  nden[5] = 9.998878241e-11
  nden[6] = 8.733926903e-21
  nden[7] = 6.77124032e-21
  nden[8] = 4.776553858e-24
  nden[9] = 1.381244146e-09
  ... (showing first 10 species)

Element/charge conservation:
  C: init=1.5000000000e-09, final=1.5000000000e-09, rel_delta=-1.8073896384e-13
  H: init=1.0030000000e-06, final=1.0030000000e-06, rel_delta=6.6272094253e-13
  He: init=3.0000000000e-10, final=3.0000000000e-10, rel_delta=4.29272272

In [14]:
assert(_exit_code == 0)